In [ ]:
pip install mesa pandas matplotlib numpy

In [ ]:
#Library yang digunakan
import pandas as pd
from mesa import Agent, Model
from mesa.time import RandomActivation
import matplotlib.pyplot as plt
import os
import numpy as np

In [ ]:
#Membuat Data
jumData = 1000
np.random.seed(42) 
data = {
    'Agen_ID': range(1, jumData + 1), #Agen_ID sejumlah nilai yang sudah diinputkan
    'Umur': np.random.randint(20, 70, size=jumData),  # Umur antara 20 dan 69 tahun
    'Pendapatan': np.random.randint(1000000, 30000000, size=jumData)  # Pendapatan antara 1 juta dan 30 juta
}

df = pd.DataFrame(data)
dir=os.getcwd()
nf="mydata.xlsx"
file = os.path.join(dir, nf)
df.to_excel(file, index=False)
print(f"\nFile {nf} dengan jumlah data {jumData} telah dibuat\n")

In [ ]:
#Proses pembuatan grafik berdasarkan data yang telah digenerate
step=100

class AgentData:
    def __init__(self, agenID, umur, pendapatan):
        self.agenID = agenID
        self.umur = umur
        self.pendapatan = pendapatan

class MyAgent(Agent):
    def __init__(self, idUnik, model, data):
        super().__init__(idUnik, model)
        self.data = data

    @property
    def umur(self):
        return self.data.umur

    @property
    def pendapatan(self):
        return self.data.pendapatan

class MyModel(Model):
    def __init__(self, agen_Data):
        super().__init__()
        self.schedule = RandomActivation(self)
        self.steps = 0

        for dataRow in agen_Data:
            agent = MyAgent(dataRow.agenID, self, dataRow)
            self.schedule.add(agent)

    def step(self):
        self.schedule.step()
        self.steps += 1

    def __str__(self):
        return f"Dengan {len(self.schedule.agents)} agen dan {self.steps} langkah"

agenDataDF = pd.read_excel(file)
agenData = [AgentData(row['Agen_ID'], row['Umur'], row['Pendapatan']) for idx, row in agenDataDF.iterrows()]
model = MyModel(agenData)
for i in range(step):
    model.step()

agenUmur = [agent.umur for agent in model.schedule.agents]
agenPendapatan = [agent.pendapatan for agent in model.schedule.agents]
plt.figure(figsize=(8, 6))
plt.scatter(agenUmur, agenPendapatan, alpha=0.5, color='blue')
plt.xlabel('Umur')
plt.ylabel('Pendapatan')
plt.title(f'Agent Based Modeling\nUmur vs Pendapatan\n{model}')
plt.grid(True)
plt.show()